In [ ]:
%pip install -r requirements.txt

In [ ]:
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import pickle

from flask import Flask
from flask import request
import requests
from flask import jsonify

import os
import json
from ast import literal_eval
import traceback

application = Flask(__name__)


#загружаем модели из файла
vec = pickle.load(open("./models/tfidf.pickle", "rb"))
model = lgb.Booster(model_file='./models/lgbm_model.txt')


# тестовый вывод
@application.route("/")  
def hello():
    resp = {'message':"Hello World!"}
    
    response = jsonify(resp)
    
    return response

# предикт категории
#{"user_message":"example123rfssg gsfgfd"}
@application.route("/categoryPrediction" , methods=['GET', 'POST'])  
def registration():
    resp = {'message':'ok'
           ,'category': -1
           }

    try:
        getData = request.get_data()
        json_params = json.loads(getData) 
        
        #напишите прогноз и верните его в ответе в параметре 'prediction'



        
    except Exception as e: 
        print(e)
        resp['message'] = e
      
    response = jsonify(resp)
    
    return response

        

if __name__ == "__main__":
    port = int(os.getenv('PORT', 5000))
    application.run(debug=False, port=port, host='0.0.0.0' , threaded=True)





# Пример сохранения моделей и прогноза

In [ ]:
#сохраняем векторайзер
import pickle 

labels, levels = pd.factorize(df['category'])
df['category_int'] = labels

texts = df['message']

vec = TfidfVectorizer(max_features=100)
tfidf_mat = vec.fit_transform(texts).toarray()


#vec - модель из прошлой практики

#сохраняем модель в файл
pickle.dump(vec, open("tfidf.pickle", "wb"))

#загружаем модель из файла
vec = pickle.load(open("tfidf.pickle", "rb"))



#### сохраняем бустинг
import lightgbm as lgb
train = pd.DataFrame(tfidf_mat)
train['category_int'] = df['category_int']

train.dropna(inplace = True)

target = 'category_int'
train_columns = list(set(train.columns) - set([target]))


lgb_data_train = lgb.Dataset(train[train_columns], 
                             train[target].values, 
                             free_raw_data=False
                            )
params = {
    'objective':'multiclass', 
    'metric': 'multi_logloss',
    "num_class" : 3,
    'learning_rate': 0.01, 
    'random_state':202008,
    'subsample':0.33,
    'colsample_bytree':0.33,
    'reg_lambda':4
}

model = lgb.train(params, lgb_data_train, num_boost_round=100  )

model.save_model('lgbm_model.txt')

model = lgb.Booster(model_file='lgbm_model.txt')

#test predict
message = 'hello i want to payment but it is blocked'
model.predict(vec.transform([message]).toarray())